In [1]:
# import gensim
# model = gensim.models.KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin.gz', 
#                                                         binary=True)

# words = model.index2word

# w_rank = {}
# for i,word in enumerate(words):
#     w_rank[word] = i

# WORDS = w_rank

In [2]:
# Peter Norvig Spell Corrector
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [3]:
from google.colab import files
up = files.upload()

Saving Saraiki_corpus.csv to Saraiki_corpus.csv


In [4]:
import copy
import numpy as np
import pandas as pd
def levenshtein_edit_distance(u, v):
    # convert to lower case
    u = u.lower()
    v = v.lower()
    # base cases
    if u == v: return 0
    elif len(u) == 0: return len(v)
    elif len(v) == 0: return len(u)
    # initialize edit distance matrix
    edit_matrix = []
    # initialize two distance matrices 
    du = [0] * (len(v) + 1)
    dv = [0] * (len(v) + 1)
    # du: the previous row of edit distances
    for i in range(len(du)):
        du[i] = i
    # dv : the current row of edit distances    
    for i in range(len(u)):
        dv[0] = i + 1
        # compute cost as per algorithm
        for j in range(len(v)):
            cost = 0 if u[i] == v[j] else 1
            dv[j + 1] = min(dv[j] + 1, du[j + 1] + 1, du[j] + cost)
        # assign dv to du for next iteration
        for j in range(len(du)):
            du[j] = dv[j]
        # copy dv to the edit matrix
        edit_matrix.append(copy.copy(dv))
    # compute the final edit distance and edit matrix    
    distance = dv[len(v)]
    edit_matrix = np.array(edit_matrix)
    edit_matrix = edit_matrix.T
    edit_matrix = edit_matrix[1:,]
    edit_matrix = pd.DataFrame(data=edit_matrix,
                               index=list(v),
                               columns=list(u))
    return distance, edit_matrix

In [5]:
# LEVENSHTEIN EDIT DISTANCE
root_term = 'believe'
term = 'believeee'
# for term in terms:
edit_d, edit_m = levenshtein_edit_distance(root_term, term)
print ('Computing distance between root: {} and term: {}'.format(root_term,
                                                                    term))
print ('Levenshtein edit distance is {}'.format(edit_d))
print ('The complete edit distance matrix is depicted below')
print (edit_m)
print ('-'*30 )          

Computing distance between root: believe and term: believeee
Levenshtein edit distance is 2
The complete edit distance matrix is depicted below
   b  e  l  i  e  v  e
b  0  1  2  3  4  5  6
e  1  0  1  2  3  4  5
l  2  1  0  1  2  3  4
i  3  2  1  0  1  2  3
e  4  3  2  1  0  1  2
v  5  4  3  2  1  0  1
e  6  5  4  3  2  1  0
e  7  6  5  4  3  2  1
e  8  7  6  5  4  3  2
------------------------------


In [6]:
from nltk import edit_distance

In [7]:
edit_distance(root_term,term)

2

In [8]:
df = pd.read_csv("Saraiki_corpus.csv")
sentences_df = df[['Saraiki']]
sentences_df.head(10)

,Saraiki
0,بلوچستان دے صنعتی علاقے حب اچ واقع فیکٹری اچ گ...
1,پولیس دا آکھݨ ھا کہ صنعتی زون نظر چورنگی دے نی...
2,زخمیاں دی مرحم پٹی کیتے جام غلام قدر ہسپتال من...
3,پاکستان اچ کورونا وائرس نال متاثر تھیوݨ والیاں...
4,نیشنل کمانڈ اینڈ آپریشن سینٹر این سی او سی دے ...
5,ایندے علاوے مزید مریض بیماری دے وجہ نال دم توڑ...
6,سندھ ہزار کیسز اموات
7,گلگت بلتستان کیسز
8,ملک اچ ایں ویلے کورونا وائرس دے ھزار کیس فعال ...
9,انہاں ایں گالھ دا اظہار اپنڑیں نال منتخب ھک پر...


In [9]:
!pip install urduhack['tf']
import urduhack
urduhack.download()
from urduhack.preprocessing import remove_accents
from urduhack.normalization import remove_diacritics
sub_char = r'''ÆÁÂÂÀÅÃÄÇÐÉÊÈËÍÎÌÏÑÓÔÒ❤ØÕÖÞÚÛÙÜÝáâæàåãäçéêèðëíîìïñóôòøõößþúûùüýÿĀāĂăĄąĆćĈĉĊċČčĎďĐđĒēĔĕĖėĘęĚěĜĝĞğĠġĢģĤĥĦħĨĩĪīĬĭĮįİıĲĳĴĵĶķĸĹĺĻļĽľĿŀŁłŃńŅņŇňŊŋŌōŎŏŐőŒœŔŕŖŗŘřŚśŜŝŞşŠšŢţŤťŦŧŨũŪūŬŭŮůŰűŲųŴŵŶŷŸŹźŻżŽžſŉ$¢€£¥₩₽₹¤₱₦ƒ₮৲৳₨௹฿៛₪₫₭₲₴₵﷼≋₿ŁÐ₳§¶❡⁋※〈〉《》•…·¡¿¦¨¯´·¸º‽‼⁏※†‡ˉ˘⁇⁈⁉ˆ⁊⸘ΓΔΕΖΗΘΙΚΛΜΝΞΟΠΡΣⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩⅪⅫⅰⅱⅲⅳⅴⅵⅶⅷⅸⅹⅺⅻ⓪①②③④‱№℗℠℡☰≡Ξ㊂≣⁝⋮︙⦙⁞⋯…∷▲▼▶△▽▷⌃⌄︿﹀︽♡♢♤♧♣♦♥♠︾⌵𓏬𓃑𓏧⸬♓♒♑♐♏♎♍♌♋♊♉♈☉☽☿♀⊕♁♂♃♄♅⛢♆♇ˇ∛∜⍻☐☑☒✓✔✗✘༝྾∨√℻℀℁℅℆⅍⌚⌛☊☎☏✁✂✃✄✆✇✈✉✍✎✏✐✑✒™©®‰§¶𓍝⏎⇧⏏⌂⌘☢☣⎋⌥⌫ᵁᴴᴰ⌖⑤⑥⑦⑧⑨⑩⓵⓶⓷⓸⓹⓺⓻⓼⓽⓾⑴⑵⑶⑷⑸⑹⑺⑻⑼⑽⓿❶❷❸❹❺❻❼❽❾❿㈠㈡㈢㈣㈤㈥㈦㈧㈨㈩㊀㊁㊂㊃㊄㊅㊆㊇㊈㊉μ°℃℉㎍㎎㎏℥㏌㎚㎛㎜㎝㎞²³㎖㎗㎘㏄㏖㏒㎅㎆㎇㎈㎉㎐㎑㎒㎓㎾Ω㏑㏈㏐㏂㏘㎳㎭㏅㎪㏗
ΤΥΦΧΨΩαβγδεζηθικλμνξοπρςστυφχψωΠπ𝜫𝝅𝝥𝝿𝞟𝞹Пп∏ϖ∐ℼㄇ兀「」『』【】〔〕︵︶︷︸︹︺︻︼︽︾︿﹀﹁﹂﹃﹄½⅓⅔¼¾⅕⅖⅗⅘⅙⅚↕↖↗↘↙↚↛↜↝↞↟↠↡↢↣↤↥↦↧↨↩↪↫↬↭↮↯↰↱↲↳↴↶↷↸↹↺↻⟳↼↽↾↿⇀⇁⇂⇃⇄⇅⇆⇇⇈⇉⇊⇋⇌⇍⇎⇏⇕⇖⇗⇘⇙⇚⇛⇜⇝⇞⇟⇠⇡⇢⇣⇤⇥⇦⇧⇨⇩⇪⌅⌆⌤⏎▶☇☈➔➘➙➚➛➜➝➞➟➠➡➢➣➤➥➦➧➨➩➪➫➬➭➮➯ღ•⁂€™↑→↓⇝√∞░▲▶◀●☀☁☂☃☄★☆☉☐☑☎☚☛☜☝☞☟☠☢☣☪☮☯♔♕♖♗♘♙♚♛♜♝♞♟♩♪♫♬♭♮♯𝄞𝄡𝄢𝄪𝄫🎵🎶🎼𓏢ϟ☀☁☂☃☄☉☼☽☾♁♨❄❅❆༄࿓☸☹☺☻☼☽☾♔♕♖♗♘♚♛♜♝♞♟♡♨♩♪♫♬✈✉✍✎✓✔✘✚✞✟✠✡✦✧✩✪✮✯✹✿❀❁❂❄❅❆❝❞❣❤❥❦➤➱➲➳➴➵➶➷➸➹➺➻➼➽➾⤴⤵↵↓↔←→↑⍇⍈⍐⍗⇫⇬⇭⇮⇯⇰⇱⇲⇳⇴⇵⇶⇷⇸⇹⇺⇻⇼⇽⇾⇿⇐⇑⇒⇓⇔⅐⅛⅜⅝⅞⅑⅒↉⅟∟∬∭∮∵∷≦≧≋╳✕✖✚﹢﹣﹤﹥％＋－／＝∧∠∩∪°÷≡≥∞∫≤≠∨‰‱⊥π±√∑∴×¬﹙﹚﹛﹜﹝﹞﹤﹥（）＜＞｛｝❬❭❮❯❰❱〖〗〘〙〚〛⟨⟩‹›«»｢｣⌃⌄⌵«»‘’‚‛“”„‟‹›❛❜❝❞〝〞〟〃＂＇′″`ˊ―—–'''
a_special = [i for i in sub_char]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 105 kB 5.0 MB/s 
     |████████████████████████████████| 3.4 MB 24.7 MB/s 
     |████████████████████████████████| 1.1 MB 43.8 MB/s 
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1
98304/89078 [=================================] - 0s 0us/step


In [10]:

def combine_word(curr):
  temp = ''
  sample_str=''
  guess = "و"
  indices = []
  occurrences = curr.count(guess)
  # print(occurrences)
  for i, a in enumerate(curr):
    if i!=0 and i!=len(curr)-1:
      if a == guess:
        if curr[i-1]==" " and curr[i+1]==" ":
          indices.append(i)
  if len(indices)==1:
    n = indices[0]
    replacement = "_و_"
    # Replace character at nth index position
    sample_str = curr[0:n-1] + replacement + curr[n+2: ]
    return sample_str
  elif len(indices)>1:
    for i in range(len(indices)):
      n=indices[i]
      # print(n)
      replacement = "_و_"
      # print(i)
      if i==0:
        sample_str = curr[0:n-1] + replacement + curr[n+2:]       
      else:
        sample_str = sample_str[0:n-1] + replacement + sample_str[n+2:]
      if i==len(indices)-1:
        return sample_str
        # break
  else:
    return curr


In [11]:
def clean_dia_acce(text):
  cleared_diacritics_accents  = []
  for i in text:
    if i!='':
      ac = remove_accents(i)
      di = remove_diacritics(ac)
      cleared_diacritics_accents.append(di)
  return cleared_diacritics_accents 

In [12]:
import re
# text cleaning and pre-processing:
def delete_urdu_english_symbols2(sentence):
      text = re.sub(r"\d+", " ", sentence)
        # English punctuations
      text = re.sub(r"""[!"#$%&'()*+—,-./:;<=>?@[\]^_`{|}~]+""", " ", text)
        # replacing ءچ
      char = r'ءچ'
      text = re.sub(char,"اچ", text)
      char = r'دء'
      text = re.sub(char,"دے", text)
        # Urdu punctuations
      text = re.sub(r"[“”’ء۔؟‘٭،]+", " ", text)
      emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
      text = emoji_pattern.sub(r'', text)
        # Arabic numbers
      text = re.sub(r"[٠‎١‎٢‎٣‎٤‎٥‎٦‎٧‎٨‎٩]+", " ", text)
        # text = re.sub(r"[^\w\s]", " ", text)
        # Remove English characters and numbers.
      text = re.sub(r"[a-zA-z0-9]+", " ", text)
        # remove remove_accents
        # text = re.sub(r'[]+')
        # remove multiple spaces.
      text = re.sub(r" +", " ", text)
      text = text.split(" ")
        # some stupid empty tokens should be removed.
      text = [t.strip() for t in text if t.strip()]
      return " ".join(text)

In [17]:
# !pip install --upgrade spacy
from spacy.lang.ur import Urdu
nlp = Urdu()
nlp.add_pipe('sentencizer')
# d = 'فائربریگیڈ دی چار گڈیاں نے ݙو گھنٹے دی کوششاں دے بعد بھا تے قابو پا گھیدا بھا اتنی ھائی کہ پرے توں بھا دی لٹیاں ݙکھائی ݙیندیاں پیاں ھن ریجنل کنٹرول سینٹر جامشور دے مطابق ھک سو بتری کے وی دے ٹرانسفارمر اچ اسپارک دی وجہہ نال دھماکے تھئے تے بھا بھڑکی ۔ ترجمان کے الیکٹرک دے مطابق نیشنل گرڈ توں بجلی دی بحال دے پاروں کراچی دی بجلی دی صورتحال اچ قدرے بہتری آئی ھ ھن میگا واٹ بجلی فراہم کیتی ویندی پعی ھ رھائش علاقیاں اچ لوڈ منیجمنٹ وی نی کیتی ویندی پئی ۔'
sentences = []
# for i in cleaned_sentences:
for i in df['Saraiki']:
  if i!='' and i!=' ' and i!='\n' and i!='\t':
    doc = nlp(str(i))
    for sent in doc.sents:
      if str(sent).strip()!='' and str(sent).strip()!=' '  and str(sent).strip()!='\n'  and str(sent).strip()!='\t':
        sentences.append([delete_urdu_english_symbols2(str(sent).strip())])

In [18]:
cleaned_sentences = [[combine_word(j)] for i in sentences for j in clean_dia_acce(i)]
sentences = cleaned_sentences

In [19]:
sentences

[['بلوچستان دے صنعتی علاقے حب اچ واقع فیکٹری اچ گیس لیکیج نال دھماکے دے نتیجے اچ مزدور سڑ تے زخمی تھئ گئے'],
 ['پولیس دا آکھݨ ھا کہ صنعتی زون نظر چورنگی دے نیڑے واقع این پی کاٹن فیکٹری ڊی گیس لائن اچ صفائی دے دوران دھماکا تھئے جیندے نال مزدور زکمی تھئ گئے'],
 ['زخمیاں دی مرحم پٹی کیتے جام غلام قدر ہسپتال منتقل کر ݙیتا گئے ہسپتال دے ایم ایس ڈاکٹر عباس لاسی نے ݙسایا کہ دھماکے نال سڑن والے مزدوراں کوں ہسپتال گھن تے آیا گئے جئیں اچون چار دی حالت صحیح کینی انہاں دا آکھݨ ھا کہ تمام مزدوراں کون طبی امداد ݙیوݨ دے بعد کراچی دے سول ہسپتال منتقل کر ݙیتا گئیا ھ ایں تو پہل بلوچستان دے گھراں اچ گیس لیکج دی وجہہ نال مرݨ دے کافی واقعے سامنڑیں آ چکے ھن ایں مہینے دے آغاز اچ کوئٹہ دے عبدالرزاق ٹاون دے ھک گھر اچ گیس دی لیکیج دی وجہہ نال دھماکا تھیا ھا جیندے نتیجے اچ ڈو عورتاں زخمی تھئیاں ھن جنہاں کوں ہسپتال منتقل کر ݙیتا گئیا ھا'],
 ['پاکستان اچ کورونا وائرس نال متاثر تھیوݨ والیاں کیتے قرنطینہ دی مدت گھٹا ڈیتی گئی گیساں اچ ودھاوا تھیندا پئے گھنٹیاں دے دوران مزید ہزار بندے ایندا شکار تھے ھن'],
 ['نیشنل کما

In [20]:
!pip install emoji
import emoji
word_list = []
from emoji import UNICODE_EMOJI
stop_words = ['۔','؟']
for i in sentences:
  for j in i:
    if j!='' and j!='\n' and j!=' ' and j!='\t' and j!=None:
      doc = nlp(j)
      word_list.append([word.text for word in doc if word.text not in stop_words and word.text not in UNICODE_EMOJI])
# for i, j in enumerate(word_list):
#   for k,l in enumerate(j):
#     if l=='ھ':
#       print(i,k) #list index and inside list item index
#       word_list[i][k] = 'ہے'
for i, j in enumerate(word_list):
  for k,l in enumerate(j):
    if l=='ھ۔':
      print(i,k) #list index and inside list item index
      word_list[i][k] = 'ہے۔'
# for i,j in enumerate(word_list):
#   if j!=None:
#     for k,l in enumerate(j):
#       word_list[i][k]=word_list[i][k].replace("_"," ")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 5.2 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=716cc3ea5313d14d2653bf148a4ab16a57b8791dcb9075dbb7c46701d5d00b0b
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [21]:
import itertools
def get_plain_vocabluary():
    sentencess = [[word.text for word in nlp(j)] for sentence in sentences for j in sentence]
    mergesentences = list(itertools.chain.from_iterable(sentencess))
    plainvocabulary = list(set(mergesentences))
    return plainvocabulary

In [22]:
s = get_plain_vocabluary()

In [23]:
s

['دگرگوں',
 'مساتیں',
 'پلدی',
 'مولاحسین',
 'پنجھواوݨ',
 'سلائی',
 'ٹرکیا',
 'لنی',
 'یونیورسٹیں',
 'البلدان',
 'داراجنپور',
 'لابي',
 'بدلیجا',
 'خشکال',
 'ںبے',
 'ݙینہارے',
 'ݙبرا',
 'قریبا',
 'ݨ',
 'ایہوبیانیہ',
 'مانگی',
 'ونڈن',
 'ابتداے',
 'سدھائے',
 'ایڈورٹائزمنٹ',
 'باسربیسکا',
 'تانگاں',
 'مکینکس',
 'تھیونڑں',
 'میور',
 'اڳواٹوں',
 'کائنمی',
 'ٹٻڑیں',
 'لمحے',
 'شیئے',
 'لڳدناو',
 'سرنھ',
 'صوبہہوا',
 'جائززہ',
 'جسیم',
 'غٹکاوݨ',
 'ور',
 'سلہاڑیں',
 'عزر',
 'دوارے',
 'میانخیل',
 'اللہ',
 'ٹھٹھہ',
 'ریمارنڈ',
 'کتھڑ',
 'شخصیات_و_تذکرہ',
 'اردوگاؤں',
 'گوارا',
 'تعلیم_و_تربیت',
 'ماڈیڈی',
 'آکڑی',
 'کاجیا',
 'کاروبارے',
 'عزتارݨ',
 'کلازنگ',
 'ڈیندےمیڈی',
 'پاڳئے',
 'ھےسݨیا',
 'چن',
 'ݙچکݨ',
 'سنس',
 'کھڑالا',
 'کٹیے',
 'قرقیسیا',
 'متاخر',
 'جونوڑا',
 'کےتلوؤں',
 'اقوام',
 'سنجیدگی',
 'پھین',
 'کھینڈوکھانڈ',
 'بدھے',
 'اوائے',
 'اگھسی',
 'سوئزرلینڈ',
 'قدسی',
 'ہسنی',
 'لینڈس',
 'چرھ',
 'یرایکل',
 'سکھݨا',
 'عملدرآمد',
 'کیفر',
 'اساسی',
 'سنڄاݨوسیب',
 'تعریفیں',
 'بالراں',
 

In [24]:
def levenshtein_distance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [25]:
voc_words = list(set(itertools.chain.from_iterable([get_plain_vocabluary()])))

In [26]:
voc_words

['دگرگوں',
 'مساتیں',
 'پلدی',
 'مولاحسین',
 'پنجھواوݨ',
 'ٹرکیا',
 'سلائی',
 'لنی',
 'یونیورسٹیں',
 'البلدان',
 'داراجنپور',
 'لابي',
 'بدلیجا',
 'خشکال',
 'ںبے',
 'ݙینہارے',
 'ݙبرا',
 'قریبا',
 'ݨ',
 'ایہوبیانیہ',
 'مانگی',
 'ونڈن',
 'ابتداے',
 'سدھائے',
 'ایڈورٹائزمنٹ',
 'باسربیسکا',
 'تانگاں',
 'مکینکس',
 'تھیونڑں',
 'میور',
 'اڳواٹوں',
 'کائنمی',
 'ٹٻڑیں',
 'لمحے',
 'شیئے',
 'لڳدناو',
 'سرنھ',
 'صوبہہوا',
 'جائززہ',
 'جسیم',
 'غٹکاوݨ',
 'ور',
 'سلہاڑیں',
 'عزر',
 'دوارے',
 'میانخیل',
 'اللہ',
 'ٹھٹھہ',
 'ریمارنڈ',
 'کتھڑ',
 'شخصیات_و_تذکرہ',
 'اردوگاؤں',
 'گوارا',
 'تعلیم_و_تربیت',
 'ماڈیڈی',
 'آکڑی',
 'کاجیا',
 'کاروبارے',
 'عزتارݨ',
 'کلازنگ',
 'ڈیندےمیڈی',
 'پاڳئے',
 'ھےسݨیا',
 'چن',
 'ݙچکݨ',
 'سنس',
 'کھڑالا',
 'کٹیے',
 'قرقیسیا',
 'متاخر',
 'جونوڑا',
 'کےتلوؤں',
 'اقوام',
 'سنجیدگی',
 'پھین',
 'کھینڈوکھانڈ',
 'بدھے',
 'اوائے',
 'اگھسی',
 'سوئزرلینڈ',
 'قدسی',
 'ہسنی',
 'لینڈس',
 'چرھ',
 'یرایکل',
 'سکھݨا',
 'عملدرآمد',
 'کیفر',
 'اساسی',
 'سنڄاݨوسیب',
 'تعریفیں',
 'بالراں',
 

In [27]:
def spelling_correction(sentence):
    doc = nlp(sentence)
    splittedsentence = [word.text for word in doc]
    vocwords = voc_words
    for i,word in enumerate(splittedsentence):
        if (word not in vocwords and not word.isdigit()): # ignore digits
            levdistances = []
            for vocword in vocwords:
                levdistances.append(levenshtein_distance(word,vocword))
            splittedsentence[i] = vocwords[levdistances.index(min(levdistances))]
            # print(levdistances)
        else:
            splittedsentence[i] = word
    return ' '.join(splittedsentence)

In [28]:
# 1 word is not spelled correctly "lihgts"
print(spelling_correction("بلوچستاچ دے صنعتی علاقے حب اچ واقع فیکٹر اچ")) #فیکٹری

بلوچستان دے صنعتی علاقے حب اچ واقع فیکٹر اچ


In [29]:
print(spelling_correction("بلوچستافیص")) #فیکٹری

بلوچستانی


In [30]:
#,اسپیکر کریندے جمہوریت سرپرست بچاونڑاں
para = '''
وزیر اعظم عمران خان نے ڈپٹی اسپکر قاسم خان سوری دی تحریک عدم اعتماد مسترد کرݨ دیخلاف سپریم کورٹ دے فیصلے تے مایوسی دا اظہار کریند ہووے آکھیا کہ امپورٹڈ حکومت کوں تسلیم نہ کریساں تے عوام میں نکلساں اتوار کوں احتجاج دی کال ڈے ڈیتی

قوم نال گالھ مہاڑ کریندے ہووے آکھیا کہ ٢٦ سال پہل توں ٹرݨ والی تحریک انصاف دے کڈھیں اصول تبدیل نی تھئے ، خوداری ، تحریک دا ناں انصاف تے ول فلاحی ریاست تے انسانی تھائی ، جئیں ویلے پارٹی شروع کیتی انہاں تریھ اصولاں تے ٹُرا ھاں۔

انہاں آکھیا کہ سپریم کورٹ دے فیصلے تے میکوں مایوسی تھئی ھِ لیکن واضح کرینداں کہ سپریم کورٹ تے پاکستان دی عدلیہ دی میں عزت کریندا ھاں میں ھک واری جیل گزاری ھِ او آزاد عدلیہ دی تحریک دوران جیل گئیاں ، کہیں وی معاشرے دی بنیاد تے انصاف تے ہوندی ھِ تے انصاف دی رپرست عدالت ہوندی ھِ

انہاں آکھیا کہ عدلیہ جو وی فیصلہ کریندی ھِ تسلیم کریندے ھیں تے افسوس ایں پاروں تھیئےء کیونکہ ڈپٹی اسپیکر نے تحریک ایں پاروں روکی ھائی کیونکہ عدم اعتماد اچ آرٹیکل ٥ یعنی بیرونی مداخلت تھئی ھائی تے میں اے چاھنداں ھم کہ سپریم کورٹ گھٹ توں گھٹ ایکوں ڈیکھداں تاں صحیح ، اے ھک وڈا سنجیدہ الزام ھِ کہ باہروں ھک ملک تساڈے اندر سازش کر تے حکومت ڈھاندا ھِ ۔ انہاں آکھیا کہ میں چاھنداں ھم کہ ایندی تحقیقات تھیووے ، سپریم کورٹ گھٹ توں گھٹ جو مراسلہ ایکوں طلب کر تے ڈیکھ گھیندا ھِ جو اساں آکھیندے پئے ھیں اتنی وڈی سازش ھِ حکومت تبدیل کرݨ دی کوشش ھِ اساں کیا سچ بولیندے پئے ہیں یا کینا، ایں تے میکوں مایوسی تھئی کیونکہ اے اتنا وڈا سنجیدہ مسئلہ ھا تے ایں تے سپریم کورٹ اچ کوئی گاھ نی کیتی ۔ انہاں دا آکھیا کہ ڈوہا افسوس اے ھِ کہ اتنا جلدی فیصلہ آیا تے ٦١ اے یعنی کھلے عام ھارس ٹریڈنگ تھیندی رہے ھِ سیاست داناں دے ضمیر خریدے ویندے رھائے بھیڈ بکریاں وانگ ہوٹلاں اچ بند کیتا ویندا رھیا ھِ تے انہاں دی قیمتاں لایا گئیاں ھن۔ وزیر اعظم نے آکھیا کہ سوشل میڈیا دا زمانہ ھِ بال بال کوں پتہ ھِ کہ کیں قیمت تے اپنڑیں ضمیر ویچندے پئے ھن کیھڑی جمہوریت ھِ تے دنیا دی کیھڑی ایہو جئی مہوریت اچ ایندی اجازت ڈیتی ویندی ھِ

انہاں قوم کوں مخاطب کریندے ہووے آکھیا کہ قوم کوں آکھینداں کہ جئیں ویلے باھروں ساز تھیندی پئی ہووے حکومت ڈھاندی پئی ہووے تاں تساں اپنڑیں آپ کو بچاونڑاں ھِ ، تساں ایندے خلاف کھڑے نا تھیسوں تاں کہیں بئے نے تساں کوں نی بچاونڑا ، بدی دیخلاف قوم کھڑی تھیندی ھِ

'''

In [31]:
print(spelling_correction("بلوچستافیص")) #فیکٹری

بلوچستانی


In [32]:
ds = nlp(para)
for i in ds.sents:
  print(spelling_correction(i.text))

ݨ وزیر اعظم عمران خان نے ڈپٹی اسپکر قاسم خان سوری دی تحریک عدم اعتماد مسترد کرݨ دیخلاف سپریم کورٹ دے فیصلے تے مایوسی دا اظہار کریند ہووے آکھیا کہ امپورٹر حکومت کوں تسلیم نہ کریساں تے عوام میں نکلسوں اتوار کوں احتجاج دی کال ڈے ڈیتی ݨ قوم نال گالھ مہاڑ کریندے ہووے آکھیا کہ ٢٦ سال پہل توں ٹرݨ والی تحریک انصاف دے کڈھیں اصول تبدیل نی تھئے ݨ خوداری ݨ تحریک دا ناں انصاف تے ول فلاحی ریاست تے انسانی تھائی ݨ جئیں ویلے پارٹی شروع کیتی انہاں تریھ اصولاں تے ٹرا ھاں ݨ
ݨ انہاں آکھیا کہ سپریم کورٹ دے فیصلے تے میکوں مایوسی تھئی ھر لیکن واضح کرینداں کہ سپریم کورٹ تے پاکستان دی عدلیہ دی میں عزت کریندا ھاں میں ھک واری جیل گزاری ھر او آزاد عدلیہ دی تحریک دوران جیل گئیاں ݨ کہیں وی معاشرے دی بنیاد تے انصاف تے ہوندی ھر تے انصاف دی پرست عدالت ہوندی ھر ݨ انہاں آکھیا کہ عدلیہ جو وی فیصلہ کریندی ھر تسلیم کریندے ھیں تے افسوس ایں پاروں تھیئے کیونکہ ڈپٹی اسپیکر نے تحریک ایں پاروں روکی ھائی کیونکہ عدم اعتماد اچ آرٹیکل ٥ یعنی بیرونی مداخلت تھئی ھائی تے میں اے چاھنداں ھم کہ سپریم کورٹ گھٹ توں گھٹ ایکوں ڈیکھساں تاں صحیح

# Fill in the Blanks

In [ ]:
[" ".join(word_list[0])]


['بلوچستان دے صنعتی علاقے حب اچ واقع فیکٹری اچ گیس لیکیج نال دھماکے دے نتیجے اچ مزدور سڑ تے زخمی تھئ گئے']

In [ ]:
fill= [" ".join(i) for i in word_list[:50000] if len([word.text for word in nlp(" ".join(i))])>=20 and len([word.text for word in nlp(" ".join(i))])<=100]
# fill = fill[:200]
# with open('fill_in_blanks.txt', 'w') as f:
#     for item in fill:
#         f.write(item+"\n")

In [ ]:
len(fill)

14427

In [ ]:
lst = []
vc = []
import random
l = 0
m = 0 
mask_text = []
for i in fill:
  d = [word.text for word in nlp(i)]
  c = random.choice([i for i in range(2,len(d))])
  word = d[c]
  d[c] = '؟'
  if d[-1]=='؟':
    if l<100:
      lst.append(" ".join(d))
      vc.append(word)
      mask_text.append(" ".join(d))
      l+=1
  else:
    if m<100:
      lst.append(" ".join(d))
      vc.append(word)
      m+=1
      mask_text.append(" ".join(d))
df2 = pd.DataFrame({'Sentence':lst,'Target Word':vc})
df2.to_csv('sentence_word_error.csv',index=False)


In [ ]:
len(df2)

200

In [ ]:
for i in df2['Target Word']:
  if i=='؟':
    print('yes')

In [ ]:
lst[47]

'بلوچستان دے صنعتی علاقے حب اچ واقع فیکٹری اچ گیس لیکیج نال دھماکے دے نتیجے اچ مزدور سڑ تے زخمی ؟ گئے'

In [ ]:
with open('fill_in_blanks.txt', 'w') as f:
    for item in mask_text:
        f.write(item+"\n")

In [ ]:
d = 'عمران خان دی اڳواݨی MASK آندا الیکشن بھاری'
import spacy
context_words= []
nlp = spacy.blank('ur')
all = []
for i in mask_text:
  doc = nlp(i)
  tokens = [token.text for token in doc]
  all.append(tokens)
k = 0
for i in mask_text:
  doc = nlp(i)
  tokens = [token.text for token in doc]
  
  for i,j in enumerate(tokens):
    if i>=2 and i<len(tokens)-2 and j=='؟':
          print(tokens)
          context_w = [tokens[i-2],tokens[i-1],tokens[i+1],tokens[i+2]]
          context_words.append(context_w)
    if i==len(tokens)-1 and j=='؟':
          print(tokens)
          context_w = [tokens[i-2],tokens[i-1]]
          context_words.append(context_w)
        # elif i==1 and j=='MASK':
        #   # print(tokens)
        #   context_w = [tokens[i-3],tokens[i-2],tokens[i-1],tokens[i+1]]
        #   context_words.append(context_w)
        # elif i==2 and j=='MASK':
        #   # print(tokens)
        #   context_w = [tokens[len(tokens)-2],tokens[len(tokens)-1],tokens[i-2],tokens[i-1]]
        #   context_words.append(context_w)
        # elif i==len(tokens)-1 and j=='MASK':
        #   # print(tokens)
        #   context_w = [tokens[len(tokens)-3],tokens[len(tokens)-2],tokens[len(tokens)-i-1],tokens[len(tokens)-i]]
        #   context_words.append(context_w)
        # elif i==len(tokens)-2 and j=='MASK':
        #   # print(tokens)
        #   context_w = [tokens[len(tokens)-3],tokens[len(tokens)-1],tokens[len(tokens)-i-2],tokens[len(tokens)-i-1]]
        #   context_words.append(context_w)
        # elif j=='MASK':
        #   context_w = [tokens[-2],tokens[-1],tokens[0],tokens[1]]
        #   context_words.append(context_w)
  k+=1        
context_words

['وزیر', 'اعظم', 'عمران', 'خان', 'قوم', 'نال', 'گالھ', 'مہاڑ', 'کریندے', 'ہووے', 'اکھیا', 'کہ', 'کافی', 'ویلے', 'توں', 'میں', 'تساں', 'نال', 'گالھ', 'مہاڑ', 'نی', 'کر', 'سکا', 'اصولا', 'تاں', 'میکوں', 'پارلیمنٹ', 'اچ', 'اے', 'گالھ', 'کرنی', 'چاھیدے', 'ہے', 'لیکن', 'بدقسمی', 'نال', 'پارلیمنٹ', 'اچ', 'اپویشن', 'گالھ', 'نی', 'کرن', 'ڈیندی', 'انہاں', 'اکھیا', 'کہ', 'پارلیمنٹ', 'اچ', 'این', 'ار', 'او', 'دی', 'تانگ', 'اچ', 'لوگ', 'بیٹھے', 'ھن', 'انہاں', 'دے', 'مطالبات', 'پورے', 'نہ', 'کرو', 'تاں', 'او', 'گالھ', 'ہی', '؟', 'کرن', 'ڈیندے']
['انہاں', 'اکھیا', 'کہ', 'جئیں', 'مقصد', 'پاروں', 'اساں', 'ملک', 'بنڑایا', 'ھا', 'جئیں', 'ویلے', 'تک', 'ایں', 'مقصد', 'تے', 'عمل', 'نہ', 'کریسوں', 'اساں', 'ترقی', 'نی', 'کر', 'سکدی', 'مہنگائی', 'دے', 'پاروں', 'سوال', 'دا', 'جواب', 'ڈیندے', 'ہووے', 'وزیر', 'اعظم', 'اکھیا', 'کہ', 'میکوں', 'احساس', 'ہے', 'ای', 'ایہو', 'جئی', 'شہ', 'ہے', 'جو', 'میکوں', 'کئی', 'واری', 'راتاں', 'کوں', 'جگا', 'ڈیندی', 'ہے', 'اساں', 'ماضئ', 'دی', 'حکومتاں', 'دا', 'چھوڑا', 'ہوھیا', '

[['گالھ', 'ہی', 'کرن', 'ڈیندے'],
 ['نال', 'اضافہ'],
 ['دی', 'وجہ', 'معیشت', 'کوں'],
 ['ودھیے', 'اتے', 'توں', 'ودھ'],
 ['تے', 'کہیں', 'نئیں', 'کیتی'],
 ['حلفی', 'دی', 'کھلݨ', 'ہک'],
 ['سال', 'پی', 'وی', 'دا'],
 ['مہاڑ', 'نی', 'سکا', 'اصولا'],
 ['تاریخ', 'دا', 'تون', 'وڈا'],
 ['سیکوریٹی', 'دے', 'پروف', 'انتظامات'],
 ['نال', 'پارلیمنٹ', 'اپویشن', 'گالھ'],
 ['خسارہ', 'ملے', 'اکھیا', 'کہ'],
 ['بہتر', 'تھیندی', 'اے', 'ٹیکس'],
 ['گو', 'جی', 'پی', 'شرح'],
 ['لندن', 'ونڄ', 'رانا', 'شمیم'],
 ['اچ', 'پاکستانی', 'طیفا', 'ان'],
 ['ویلے', 'توں', 'تساں', 'نال'],
 ['ہے', 'ای', 'جئی', 'شہ'],
 ['بنڑدے', 'کہ', 'عوام', 'دی'],
 ['انتظام', 'سانڳے', 'دی', 'اڳواݨی'],
 ['ٻݨاوݨا', 'ہوسی', 'اچ', 'ایئر'],
 ['دی', 'شہیں', 'دے', 'خلاف'],
 ['اے', 'پی', 'صدر', 'مملکت'],
 ['افسوس', 'کیتے', 'کوں', 'آپݨے'],
 ['دے', 'مطابق', 'اچ', 'ضلع'],
 ['نامزدگی', 'تے', 'ݙتی', 'اتے'],
 ['این', 'ار', 'دی', 'تانگ'],
 ['احساس', 'ہے', 'ایہو', 'جئی'],
 ['مساجد', 'دے', 'دے', 'موقع'],
 ['سبزیاں', 'دی', 'ودھاوݨ', 'تے'],
 ['ابوظہبی', 'اتے', '

# Tokenizer

In [ ]:
s = '''اسلام آباد (اے پی پی)پاکستان تحریک انصاف دے رہنما فواد چوہدری آکھیے جیڑھا پیپلزپارٹی سپریم کورٹ توں عبوری حکم امتناع منگیا ہا عدالت ایکوں مسترد کرݙتے اتے ڈپٹی سپیکر قاسم سوری دی رولنگ برقرار رہسی۔ سپریم کورٹ دے ٻاہرو پی ٹی آئی رہنماواں سینٹر فیصل جاوید، عام ڈوگر، شوکت بسرا اتے ٻئے رہنماں نال الیندے فواد چوہدری آکھیا جو آرٹیکل 69 دے تحت سپریم کورٹ کوں اختیار نئیں جو او پارلیمنٹ دی کارروائی تے ڳالھ کرے، میں سیاسی جماعتاں کوں آہداں جو او کیوں ہݨ شیر نئیں ٻݨدے، اے روندے ودن ڄݙاں جو حکومت سینہ ٹھوک کے ٻاھرو آئی اے، مسلم لیگ ن، پیپلزپارٹی اتے جے یو آئی تاں چاہندے ہن جو حکومت لڳی ونڄے، ڄݙاں حکومت نئین رہی تاں اے عدالت آڳین کہ حکومت کوں بحال کرو، سیاست دے فیصلے عوام اچ تھیندن۔ انہاں چیلنج کیتے جو الیکشن دو آون، پاکستان دے لوک نہ پنݨے ہین نا بے غیرت، اے تمغے تساں آپݨے کیتے ہی رکھو، عمران خان دی اڳواݨی اچ آندا الیکشن بھاری مینڈیٹ نال جیتسوں۔'''

# Stop Words

In [ ]:
for i in range(len(s)):
  if s[i]=='۔' or s[i]=='؟' or s[i]=='،':
    for num in reversed(range(i)):
      if s[num] == ' ':
        print(s[num+1:i])
        break


رہسی
جاوید
ڈوگر
کرے
ٻݨدے
اے
ن
ونڄے
کرو
تھیندن
آون
غیرت
آپݨے
رکھو
جیتسوں


In [ ]:
## 1. Select random words 
## 2. Replace chars randomly 1,2,3,4,5
## 3. Predict using ld
## 4. Calculate average distance w.r.t chars

In [ ]:
unq_words = set()
for i in word_list:
  for j in i: 
    if j not in unq_words:
      unq_words.add(j)

In [ ]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
dd = set()
for i in tokenizer.get_vocab().keys():
  if i not in dd:
    dd.add(i)
pure_saraiki_vocab = unq_words.difference(dd)
len(pure_saraiki_vocab)
pure_saraiki_vocab = list(pure_saraiki_vocab)
saraiki_alpha_bets=['ق','و','ع','ر','ت','ے','ی','ہ','پ','ا','س','د','ف','گ','ح','ج','ک','ل','ز','ش','چ','ط','ب','ن','م','ڑ','ٹ','ئ','ۃ','آ','ص','ڈ','غ','ھ','ض','خ','ذ','ژ','ث','ظ','ں','ي','ڃ','ڋ','_','ڳ','ؤ','ئ','ڑ','ٹ','ۃ','آ','ص','ڈ','ݙ','غ','ح','ڄ','خ','ض','ذ','ژ','ث','ظ','ٻ','ں','ݨ','ق','و','ع','ر','ت','ے','ء','ی','ہ','پ','ا','س','د','ف','گ','ھ','ج','ک','ﺋ','ل','ز','ش','چ','ط','ه','ب','ن','م','ن٘','ڊ']

for i,j in enumerate(pure_saraiki_vocab):
  for k in j:
    if k not in  saraiki_alpha_bets:
      print(j)
      pure_saraiki_vocab.pop(i)
    # if k not in saraiki_alpha_bets:
    #   
    #   print(j)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
     |████████████████████████████████| 6.6 MB 32.3 MB/s 
     |████████████████████████████████| 596 kB 56.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.
ــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــسئیں
ــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــسئیں
ــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــسئیں
ــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ

In [ ]:
import random
word_longer_than_5 = [i for i in list(pure_saraiki_vocab) if len(i)>=5]
sampled_list = random.sample(list(word_longer_than_5), 100)
print(sampled_list)
df4 = pd.DataFrame({'Saraiki Words':sampled_list})

['بھانسالی', 'کشتیہ', 'ورڈپریسنگ', 'بھنڳور', 'کھیڑے', 'لهامو', 'مقبولاں', 'ویروا', 'کھیندواݨ', 'کرناٹک', 'توھاکوں', 'جونہی', 'پریمی', 'لکھیمکیا', 'بھاندنلکھیے', 'ہیندے', 'تھیٹھر', 'سلہریا', 'منہاج', 'ااحتجاج', 'سانجھیاں', 'عابدين', 'گھیلندے', 'مونڈھ', 'ںڳئین', 'کماندو', 'جندروٹ', 'لگانا', 'ناگالینڈ', 'سیخاں', 'وآلہ_و_سلم', 'البدل', 'اوگھرا', 'اوازں', 'واقات', 'اجتماعیت', 'پیواں', 'پڑھیاں', 'مارنڑے', 'پائوث', 'حسین_و_دیگر', 'کلاسیکیت', 'گومبٹ', 'انتطامیہ', 'ہیݨاں', 'اتھرا', 'دالایا', 'ہےمچھی', 'ڈیندےسندھو', 'عابدہ', 'آوےتوں', 'ݙاکٹر', 'نارملسی', 'کڑاوݨ', 'ݙیندیں', 'چلاؤں', 'اذیتوں', 'روایتاں', 'تروکݨ', 'ملانے', 'ٻیٹھاںمیݙے', 'درباراں', 'نونٹھابوری', 'معصوم', 'سمائی', 'لتھڑی', 'آوزار', 'سپوٹر', 'ݙیکھیں', 'آفروڈ', 'عدالتیں', 'آفیسر', 'پھٹکاوݨ', 'میںاسٹیٹ', 'نبھابک', 'مرسائیں', 'ویھاڑی', 'پراندہ', 'آئےپھل', 'البرز', 'زمین_و_آسمان', 'کمسنر', 'ملایاسلام', 'اورنگزیب', 'چلینجزدا', 'ڳئےلڑھ', 'تمبیندا', 'ہائیرسیکنڈری', 'مرستیاںسارےخوشیاںدے', 'قالین', 'نظرآئی', 'ہیںایہا', 'صوبےسندھ', 'کھاویں', 'ش

In [ ]:
one_char_wrong = []
two_char_wrong = []
three_char_wrong = []
four_char_wrong = []
five_char_wrong = []
for i in df4['Saraiki Words']:
  word_sep = list(i)
  charc = random.choices(saraiki_alpha_bets,k = 1)[0]
  index = random.choices([i for i in range(len(list(word_sep)))])[0]
  word_sep[index] = charc
  one_char_wrong.append("".join(word_sep))
for i in df4['Saraiki Words']:
  word_sep = list(i)
  charc = random.choices(saraiki_alpha_bets,k = 2)
  index = random.choices([i for i in range(len(list(word_sep)))],k=len(charc))
  word_sep[index[0]] = charc[0]
  word_sep[index[1]] = charc[1]
  two_char_wrong.append("".join(word_sep))
for i in df4['Saraiki Words']:
  word_sep = list(i)
  charc = random.choices(saraiki_alpha_bets,k = 3)
  index = random.choices([i for i in range(len(list(word_sep)))],k=len(charc))
  word_sep[index[0]] = charc[0]
  word_sep[index[1]] = charc[1]
  word_sep[index[2]] = charc[2]
  three_char_wrong.append("".join(word_sep))
for i in df4['Saraiki Words']:
  word_sep = list(i)
  charc = random.choices(saraiki_alpha_bets,k = 4)
  index = random.choices([i for i in range(len(list(word_sep)))],k=len(charc))
  word_sep[index[0]] = charc[0]
  word_sep[index[1]] = charc[1]
  word_sep[index[2]] = charc[2]
  word_sep[index[3]] = charc[3]
  four_char_wrong.append("".join(word_sep))
for i in df4['Saraiki Words']:
  word_sep = list(i)
  charc = random.choices(saraiki_alpha_bets,k = 5)
  index = random.choices([i for i in range(len(list(word_sep)))],k=len(charc))
  word_sep[index[0]] = charc[0]
  word_sep[index[1]] = charc[1]
  word_sep[index[2]] = charc[2]
  word_sep[index[3]] = charc[3]
  word_sep[index[4]] = charc[4]
  five_char_wrong.append("".join(word_sep))

In [ ]:
df4['One_Character_Wrong'] = one_char_wrong
df4['One_Character_Correction'] = [spelling_correction(i) for i in df4['One_Character_Wrong']]

In [ ]:
df4['Two_Character_Wrong'] = two_char_wrong
df4['Two_Character_Correction'] = [spelling_correction(i) for i in df4['Two_Character_Wrong']]

In [ ]:
df4['Three_Character_Wrong'] = three_char_wrong
df4['Three_Character_Correction'] = [spelling_correction(i) for i in df4['Three_Character_Wrong']]

In [ ]:
df4['Four_Character_Wrong'] = four_char_wrong
df4['Four_Character_Correction'] = [spelling_correction(i) for i in df4['Four_Character_Wrong']]

In [ ]:
df4['Five_Character_Wrong'] = five_char_wrong
df4['Five_Character_Correction'] = [spelling_correction(i) for i in df4['Five_Character_Wrong']]

In [ ]:
df4

,Saraiki Words,One_Character_Wrong,One_Character_Correction,Two_Character_Wrong,Two_Character_Correction,Three_Character_Wrong,Three_Character_Correction,Four_Character_Wrong,Four_Character_Correction,Five_Character_Wrong,Five_Character_Correction
0,بھانسالی,بھانقالی,بھانسالی,بھانہالق,بھانسالی,قھانسالح,بھانسالی,لھانسالج,بھانسالی,حڃاھسوآی,جاسوسی
1,کشتیہ,کشهیہ,کشتیہ,کچپیہ,کاویہ,کڋڋیھ,کریھ,فشت_ے,فتنے,یںتٹہ,گٹہ
2,ورڈپریسنگ,ورڈپضیسنگ,ورڈپریسنگ,ارڈپریسنح,ورڈپریسنگ,ن٘رجپریسپگ,ورڈپریسنگ,شرڈپلیسنع,ورڈپریسنگ,وڳڈݨرقسٻگ,ورڈپریسنگ
3,بھنڳور,شھنڳور,بھنڳور,بآوڳور,بھنڳور,ٹھژڳݨر,ٹھڳݨ,ژھسڳؤآ,سڳآ,بڳیاوش,بیہوش
4,کھیڑے,ضھیڑے,کھیڑے,کتییے,کتیئے,کھزہے,کھوہ,جگیڳے,سگیئے,کڳݙڑے,کہڑے
...,...,...,...,...,...,...,...,...,...,...,...
95,وکیپڈیا,وکذپڈیا,وکیپڈیا,وکیپغیء,وکیپڈیا,ن٘قیپڈیخ,نہتیڈی,وکڑڑمکا,وہڑکا,وجچقڈصا,چھڈیا
96,النفسیات,ڳلنفسیات,النفسیات,هلنفسیات,النفسیات,النژن٘یان,النہیان,الݙفگیجت,النفسیات,ۃلنزسمحت,النفسیات
97,جہینہ,جہیگہ,جہینہ,ںہینو,ہینو,جہیٹژ,جیٹے,جبمنہ,جمنڑ,جۃیوڃ,جییوں
98,جماعتیں,جماعتیف,جماعتی,جماعتغں,جماعتوں,جمبحتیں,جماتیں,غحفعثیں,محفلیں,ؤظاعتےژ,ساعتاں


In [ ]:
k = 0
for i,j in zip(df4['Saraiki Words'],df4['One_Character_Correction']):
  if i==j:
    k+=1
print('One Char Accuracy:',(k/len(df4)))

One Char Accuracy: 0.79


In [ ]:
k = 0
for i,j in zip(df4['Saraiki Words'],df4['Two_Character_Correction']):
  if i==j:
    k+=1
print('Two Char Accuracy:',(k/len(df4)))

Two Char Accuracy: 0.46


In [ ]:
k = 0
for i,j in zip(df4['Saraiki Words'],df4['Three_Character_Correction']):
  if i==j:
    k+=1
print('Three Char Accuracy:',(k/len(df4)))

Three Char Accuracy: 0.39


In [ ]:
k = 0
for i,j in zip(df4['Saraiki Words'],df4['Four_Character_Correction']):
  if i==j:
    k+=1
print('Four Char Accuracy:',(k/len(df4)))

Four Char Accuracy: 0.26


In [ ]:

k = 0
for i,j in zip(df4['Saraiki Words'],df4['Five_Character_Correction']):
  if i==j:
    k+=1
print('Five Char Accuracy:',(k/len(df4)))

Five Char Accuracy: 0.17


In [ ]:
one_char_wrong_ed = []
for i,j in zip(df4['Saraiki Words'],df4['One_Character_Correction']):
  one_char_wrong_ed.append(edit_distance(i,j))
df4['One_Character_Edit_Distances'] = one_char_wrong_ed
two_char_wrong_ed = []
for i,j in zip(df4['Saraiki Words'],df4['Two_Character_Correction']):
  two_char_wrong_ed.append(edit_distance(i,j))
df4['Two_Character_Edit_Distances'] = two_char_wrong_ed
three_char_wrong_ed = []
for i,j in zip(df4['Saraiki Words'],df4['Three_Character_Correction']):
  three_char_wrong_ed.append(edit_distance(i,j))
df4['Three_Character_Edit_Distances'] = three_char_wrong_ed
four_char_wrong_ed = []
for i,j in zip(df4['Saraiki Words'],df4['Four_Character_Correction']):
  four_char_wrong_ed.append(edit_distance(i,j))
df4['Four_Character_Edit_Distances'] = four_char_wrong_ed
five_char_wrong_ed = []
for i,j in zip(df4['Saraiki Words'],df4['Five_Character_Correction']):
  five_char_wrong_ed.append(edit_distance(i,j))
df4['Five_Character_Edit_Distances'] = five_char_wrong_ed

In [ ]:
one_mean = df4["One_Character_Edit_Distances"].mean()
two_mean = df4["Two_Character_Edit_Distances"].mean()
three_mean = df4["Three_Character_Edit_Distances"].mean()
four_mean = df4["Four_Character_Edit_Distances"].mean()
five_mean = df4["Five_Character_Edit_Distances"].mean()

In [ ]:
print(one_mean)
print(two_mean)
print(three_mean)
print(four_mean)
print(five_mean)

0.32
1.01
1.78
2.46
3.39


In [ ]:
df4.head()

,Saraiki Words,One_Character_Wrong,One_Character_Correction,Two_Character_Wrong,Two_Character_Correction,Three_Character_Wrong,Three_Character_Correction,Four_Character_Wrong,Four_Character_Correction,Five_Character_Wrong,Five_Character_Correction,One_Character_Edit_Distances,Two_Character_Edit_Distances,Three_Character_Edit_Distances,Four_Character_Edit_Distances,Five_Character_Edit_Distances
0,جڑھپاں,جڑھپےں,جڑھپاں,کڑھواں,کتھواں,ئهغپاں,اپاں,ھڑڃپلں,ھےپل,جںہئاع,جوہا,0,3,3,5,4
1,خصائل,خصائپ,خصائل,خصءتل,خصلت,رصاخۃ,رواۃ,خݨشوع,خوشو,دصا_ط,دصایا,0,3,4,4,3
2,بشیری,ژشیری,بشیری,بشیذی,بشیری,بشیکخ,بچیکی,بشﺋآڊ,بشمل,گگتۃد,گفتند,0,0,2,3,5
3,الرحيم,المحيم,الرحيم,الرحڃم,الرحم,اغ_حيم,الرحيم,ڄلرحيم,الرحيم,ئفرحيچ,الرحيق,0,1,0,0,1
4,ٻائٹھ,ٻقئٹھ,ٻائٹھ,ٻاذچھ,ٻانجھ,حائٹھ,ٻائٹھ,ٻادزو,بادو,ڄانیئ,دانیں,0,2,0,4,4


In [ ]:
df4.to_csv('levenshtein_distance_corrections.csv',index=False)

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 13.5 MB/s 
     |████████████████████████████████| 6.6 MB 38.0 MB/s 
     |████████████████████████████████| 84 kB 2.7 MB/s 
     |████████████████████████████████| 596 kB 70.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

In [ ]:
len(pure_saraiki_vocab)

91989

In [ ]:
df3 = pd.DataFrame({'Saraiki':pure_saraiki_vocab})

In [ ]:
df3.to_csv('Saraiki_Vocab.csv',index=False)

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

# model = AutoModelForMaskedLM.from_pretrained("bert-base-multilingual-cased")

In [ ]:
len(tokenizer.get_vocab())

105879

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
df3 = pd.read_csv('/content/levenshtein_distance_corrections.csv')

In [34]:
df3

,Saraiki Words,One_Character_Wrong,One_Character_Correction,Two_Character_Wrong,Two_Character_Correction,Three_Character_Wrong,Three_Character_Correction,Four_Character_Wrong,Four_Character_Correction,Five_Character_Wrong,Five_Character_Correction,One_Character_Edit_Distances,Two_Character_Edit_Distances,Three_Character_Edit_Distances,Four_Character_Edit_Distances,Five_Character_Edit_Distances
0,جڑھپاں,جڑھپےں,جڑھپاں,کڑھواں,کتھواں,ئهغپاں,اپاں,ھڑڃپلں,ھےپل,جںہئاع,جوہا,0,3,3,5,4
1,خصائل,خصائپ,خصائل,خصءتل,خصلت,رصاخۃ,رواۃ,خݨشوع,خوشو,دصا_ط,دصایا,0,3,4,4,3
2,بشیری,ژشیری,بشیری,بشیذی,بشیری,بشیکخ,بچیکی,بشﺋآڊ,بشمل,گگتۃد,گفتند,0,0,2,3,5
3,الرحيم,المحيم,الرحيم,الرحڃم,الرحم,اغ_حيم,الرحيم,ڄلرحيم,الرحيم,ئفرحيچ,الرحيق,0,1,0,0,1
4,ٻائٹھ,ٻقئٹھ,ٻائٹھ,ٻاذچھ,ٻانجھ,حائٹھ,ٻائٹھ,ٻادزو,بادو,ڄانیئ,دانیں,0,2,0,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,دومتہ,دووتہ,دومتہ,دومئہ,دومتہ,دوچتئ,سوچتی,ديمشہ,تيميہ,توٻهذ,توٻڑا,0,0,3,3,4
96,پروری,پروثی,پروٹی,پروژڄ,پروین,پرذںی,پرگتی,ہرڑوف,ظروف,ہ_مسی,ہمسی,1,2,2,3,4
97,معاہدی,پعاہدی,معاہدی,معاححی,معالی,جشاددی,جاڳدی,مبائآی,لمبائی,معجصجج,ماجوج,0,2,3,4,4
98,چوٻاں,چوٻاء,چوٻاں,چوذجں,چوٹاں,چوودں,چوویں,چثےاآ,چام,سوݙزش,سوزش,0,1,2,3,4


In [61]:
from gensim.models import KeyedVectors
reloaded_word_vectors = KeyedVectors.load('/content/drive/MyDrive/Thesis2/Vectors/fasttext_skip_wind_size2_vectors_50.kv')

In [135]:
vec_size=[50,100,200,300,400]
win_size=[2,3,4,5,6]
wrong = ['One_Character_Wrong','Two_Character_Wrong','Three_Character_Wrong','Four_Character_Wrong','Five_Character_Wrong']
models = ['cbow','skip']
d = []
for i in models:
  for j in win_size:
    for k in vec_size:
      one = []
      count= 0
      reloaded_word_vectors = KeyedVectors.load('/content/drive/MyDrive/Thesis2/Vectors/fasttext_'+i+'_wind_size'+str(j)+'_vectors_'+str(k)+'.kv')
      for l in range(len(df3)):
        one.append(reloaded_word_vectors.most_similar(positive=df3['One_Character_Wrong'][l])[0][0])
      for m,n in zip(df3['Saraiki Words'],one):
        if m==n:
          count+=1
      d.append([i,str(j),str(k),1,str(count/len(df3))])
  # print(count/len(df3))

In [136]:
vec_size=[50,100,200,300,400]
win_size=[2,3,4,5,6]
wrong = ['One_Character_Wrong','Two_Character_Wrong','Three_Character_Wrong','Four_Character_Wrong','Five_Character_Wrong']
models = ['cbow','skip']
for i in models:
  for j in win_size:
    for k in vec_size:
      one = []
      count= 0
      reloaded_word_vectors = KeyedVectors.load('/content/drive/MyDrive/Thesis2/Vectors/fasttext_'+i+'_wind_size'+str(j)+'_vectors_'+str(k)+'.kv')
      for l in range(len(df3)):
        one.append(reloaded_word_vectors.most_similar(positive=df3['Two_Character_Wrong'][l])[0][0])
      for m,n in zip(df3['Saraiki Words'],one):
        if m==n:
          count+=1
      d.append([i,str(j),str(k),2,str(count/len(df3))])
  # print(count/len(df3))

In [ ]:
vec_size=[50,100,200,300,400]
win_size=[2,3,4,5,6]
wrong = ['One_Character_Wrong','Two_Character_Wrong','Three_Character_Wrong','Four_Character_Wrong','Five_Character_Wrong']
models = ['cbow','skip']
for i in models:
  for j in win_size:
    for k in vec_size:
      one = []
      count= 0
      reloaded_word_vectors = KeyedVectors.load('/content/drive/MyDrive/Thesis2/Vectors/fasttext_'+i+'_wind_size'+str(j)+'_vectors_'+str(k)+'.kv')
      for l in range(len(df3)):
        one.append(reloaded_word_vectors.most_similar(positive=df3['Three_Character_Wrong'][l])[0][0])
      for m,n in zip(df3['Saraiki Words'],one):
        if m==n:
          count+=1
      d.append([i,str(j),str(k),3,str(count/len(df3))])
  # print(count/len(df3))

In [ ]:
vec_size=[50,100,200,300,400]
win_size=[2,3,4,5,6]
wrong = ['One_Character_Wrong','Two_Character_Wrong','Three_Character_Wrong','Four_Character_Wrong','Five_Character_Wrong']
models = ['cbow','skip']
for i in models:
  for j in win_size:
    for k in vec_size:
      one = []
      count= 0
      reloaded_word_vectors = KeyedVectors.load('/content/drive/MyDrive/Thesis2/Vectors/fasttext_'+i+'_wind_size'+str(j)+'_vectors_'+str(k)+'.kv')
      for l in range(len(df3)):
        one.append(reloaded_word_vectors.most_similar(positive=df3['Four_Character_Wrong'][l])[0][0])
      for m,n in zip(df3['Saraiki Words'],one):
        if m==n:
          count+=1
      d.append([i,str(j),str(k),4,str(count/len(df3))])
  # print(count/len(df3))

In [ ]:
vec_size=[50,100,200,300,400]
win_size=[2,3,4,5,6]
wrong = ['One_Character_Wrong','Two_Character_Wrong','Three_Character_Wrong','Four_Character_Wrong','Five_Character_Wrong']
models = ['cbow','skip']
for i in models:
  for j in win_size:
    for k in vec_size:
      one = []
      count= 0
      reloaded_word_vectors = KeyedVectors.load('/content/drive/MyDrive/Thesis2/Vectors/fasttext_'+i+'_wind_size'+str(j)+'_vectors_'+str(k)+'.kv')
      for l in range(len(df3)):
        one.append(reloaded_word_vectors.most_similar(positive=df3['Five_Character_Wrong'][l])[0][0])
      for m,n in zip(df3['Saraiki Words'],one):
        if m==n:
          count+=1
      d.append([i,str(j),str(k),5,str(count/len(df3))])
  # print(count/len(df3))

In [143]:
d

[['cbow', '2', '50', 1, '0.15'],
 ['cbow', '2', '100', 1, '0.14'],
 ['skip', '2', '50', 1, '0.22'],
 ['skip', '2', '100', 1, '0.22'],
 ['cbow', '2', '50', 2, '0.1'],
 ['cbow', '2', '100', 2, '0.12'],
 ['skip', '2', '50', 2, '0.14'],
 ['skip', '2', '100', 2, '0.17']]

In [155]:
model= []
win_sizes=[]
vec_sizes=[]
chars = []
acc = []
for i in range(len(d)):
  model.append(d[i][0])
  win_sizes.append(d[i][1])
  vec_sizes.append(d[i][2])
  chars.append(d[i][3])
  acc.append(d[i][4])

In [156]:
acc

['0.15', '0.14', '0.22', '0.22', '0.1', '0.12', '0.14', '0.17']

In [157]:
df4 =pd.DataFrame({'Model':model,'Window Size':win_sizes,'Vec Size':vec_sizes,'Wrong Characters':chars,'Accuracy':acc})

In [114]:
# pd.DataFrame(list(zip(model, win_size,vec_size,chars,acc)),
#                columns =['Model', 'val'])

In [158]:
df4

,Model,Window Size,Vec Size,Wrong Characters,Accuracy
0,cbow,2,50,1,0.15
1,cbow,2,100,1,0.14
2,skip,2,50,1,0.22
3,skip,2,100,1,0.22
4,cbow,2,50,2,0.1
5,cbow,2,100,2,0.12
6,skip,2,50,2,0.14
7,skip,2,100,2,0.17


In [159]:
df4.to_csv('sim_based_spell_correction.csv',index=False)